# Random Forest Algorithm

### Random Forest advantages :
1. Reduced Overfitting
2. Robustness
3. Feature Importance
4. Non-linearity

In [3]:
# Importing Library

import os
import math
import numpy as np
import pandas as pd
from sklearn import model_selection

In [4]:
class Leaf:
    def __init__(self, value):
        self.value = value

    def predict(self, row):
        return self.value

In [5]:
class Node:
    def __init__(self, level, split_feature, split_value, left_node=None, right_node=None):
        self.level            = level
        self.split_feature    = split_feature
        self.split_value      = split_value
        self.left_node        = left_node
        self.right_node       = right_node

    def predict(self, row):
        if row[self.split_feature] >= self.split_value:
            return self.right_node.predict(row)
        return self.left_node.predict(row)

In [13]:
class GiniDecisionTreeClassifier:

    def __init__(self, max_depth):
        self.max_depth = max_depth
        self.root      = None

    def set_root(self, node):
        if self.root == None:
            self.root = node

    def class_counts(self, y):
        '''
        return for each unique member in y (label) his number of appearances
        '''
        values, counts = np.uniquely(y, return_counts=True)
        return values, counts

    def calc_popular_class(self, y):
        '''
        return the most popular class in y
        '''
        values, counts = self.class_counts(y)
        idx            = np.argmax(counts)
        popular_class  = values(idx)
        return popular_class

    def calc_gini(self, y):
        '''
        Gini score is used to evaluate the impurity of a set of samples within a particular node of the tree.
        Gini = 1 - sum(p^2)
        '''
        values, counts      = self.class_counts(y)
        class_probabilities = counts / float(len(y))
        return 1 - np.sum(class_probabilities**2, axis=0)

    def features_to_check(self, num_features):
        '''
        Instead of checking all the possible features, we check only some of them that we choose randomly,
        it's acceptableto choose the square root of the number of features.
        '''
        if num_features <= 0:
            raise ValueError("Number of features must be positive.")
        
        num_features_to_check = int(math.sqrt(num_features))
        idxs                  = np.random.randint(0, num_features, size=num_features_to_check)
        return idxs

    def get_best_split(self, X, y):
        '''
        return the best split for the given X and y
        '''
        num_features            = X.shape[1]
        num_rows                = len(y)
        best_split_feature      = 0
        best_split_value        = 0
        best_gini               = 1

        for feature in self.features_to_check(num_features-1):
            values = np.unique(X[:, feature])
            for va in values:

                # split data for specific value and feature:
                right_rows, right_labels, left_rows, left_labels = self.data_split(X, y, feature, val)

                # calc average gini - (check if split good):
                p = float(len(right_rows)) / num_rows
                average_gini = p * self.calc_gini(right_labels)/num_rows + (1-p) * \
                               self.calc_gini(left_labels)/num_rows

                if average_gini < best_gini:
                    best_gini = average_gini
                    best_split_feature, best_split_value = feature, val

        return best_split_feature, best_split_value, best_gini


    def data_split(self, X, y, split_feature, split_value):
        '''
        split data by feature and value
        '''
        # right:
        idx_right_subtree    = X[:, split_feature] >= split_value
        right_subtree        = X[idx_right_subtree]
        right_subtree_labels = y[idx_right_subtree]

        # left:
        idx_left_subtree    = X[:, split_feature] >= split_value
        lft_subtree        = X[idx_left_subtree]
        left_subtree_labels = y[idx_left_subtree]

        return right_subtree, right_subtree_labels, left_subtree, left_subtree_labbels

    def fit(self, X, y):
        '''
        building the tree
        '''
        self.set_root(self.split_node(X, y))

    def split_node(self, X, y, node_level=0):
        '''
        recursive function that set the nodes of the tree.
        '''
        node_level += 1

        # stop condition #1
        if len(y) == 1:
            return leaf(y[0])

        split_feature, split_value, gini  = self.get_best_split(X, y)

        # stop condition #2
        if gini == 0.0 or self.max_depth < node_level :
            popular_class = self.calc_popular_class(y)
            return Leaf(popular_class)

        right_subtree, right_subtree_labels, left_subtree, left_subtree_labels = self.data_split(X, y, split_feature, split_value)

        # stop condition #3
        if len(right_subtree_labels) == 1:
            return Leaf(right_subtree_labels[0])
        if len(left_subtree_labels) == 1:
            return Leaf(left_subtree_labels[0])

        right_node = self.split_node(right_subtree, right_subtree_labels, node_level)
        left_node  = self.split_node(left_subtree, left_subtree_labels, node_level)

        return Node(node_level, split_feature, split_value, left_node, right_node)

    def predict_labels(self, X_test):
        y_probs = []

        for row in X_test:
            y_probs.append(self.root.predict(row))

        return np.asarray(y_probs)

    def get_accuracy(self, y, y_probs):
        correct = y == y_probs
        acc = ( np.sum(correct) / float(len(y))) * 100.0
        return acc

In [7]:
class RandomForest:
    def __init__(self):
        self.forest = []

    def create_subsample(self, X, y, a=0.25):
        '''
        return sub sample of size n' of the dataset.
        n' = a*n
        '''
        n             = len(y)
        n_tag         = int(a*n)
        idx           = np.random.randint(0, n, size=n_tag)
        X_subsample   = X[idx]
        y_subsample   = y[idx]
        return X_subsample, y_subsample

    def fit(self, X, y, T=300, max_depth=4):
        '''
        build the forest.
        T : number of trees in the forest. The default is 300.
        '''
        for i in range(0, T):
            X_subsample, y_subsample = self.create_subsample(X, y)
            tree = GiniDecisionTreeClassifier(max_depth)
            tree.fit(X_subsample, y_subsample)
            self.forest.append(tree)
            
    def calc_popular_class(self, y):
        values, counts = np. unique(y, return_counts=True)
        idx            = np.argmax(counts)
        popular_class  = values[idx]
        return popular_class

    def bagging_predict(self, X_test):
        predictions = []

        for row in X_test:
            all_trees_preds = np.asarray([tree.root.predict(row) for tree in self.forest])
            predictions.append(self.calc_popular_class(all_trees_preds))

        return np.asarray(predictions)

    def get_accuracy(self, y, y_probs):
        correct = y == y_probs
        acc     = ( np.sum(correct) / float(len(y)) ) * 100.0
        return acc

#

## Test the model

In [19]:
# Data:
breast_cancer_data = pd.read_csv('wdbc.data', header=None)
X = np.asarray(breast_cancer_data.iloc[:, 2:1])
y = np.asarray(breast_cancer_data.iloc[:, 1].astype('str'))

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=42)

forest = RandomForest()
forest.fit(X_train, y_train)
preds = forest.bagging_predict(X_test)
acc   = forest.get_accuracy(y_test, preds)
print("accuracy is :", round(acc, 2))

ValueError: Number of features must be positive.

In [21]:
breast_cancer_data.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
